<a href="https://colab.research.google.com/github/juturpraveen/NLP-Text-Classification/blob/master/Multi_label_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sources:
https://medium.com/towards-artificial-intelligence/keras-for-multi-label-text-classification-86d194311d0e

https://datascience.stackexchange.com/questions/62086/using-lstm-for-multi-label-classification

https://machinelearningmastery.com/clean-text-machine-learning-python/

Goals:
 - Data pipeline for text classification
 - accuracy metrics

In [30]:
from google.colab import files
import io
import pandas as pd
import string

In [2]:
uploaded = files.upload()

Saving train.csv to train.csv


In [20]:
raw_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
raw_train.shape

(159571, 8)

In [21]:
raw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [22]:
raw_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [23]:
raw_train.comment_text.values

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of communit

- The text that is to be classified is the comment_text column. This needs to be cleaned and processed.
- By looking at the data punctuation, special characters like new line need to be removed.
- EDA needs to be done on statistics of the toxicity labels

In [11]:
raw_train.toxic.values.sum()

15294

In [37]:
#Clean each comment into words list
def clean_comment(comment):
  comment = comment.lower()
  comment = comment.split()
  table = comment.maketrans(' ', ' ', string.punctuation)
  comment = [w.translate(table) for w in comment]
  return comment

clean_comment("this isn't Very Good")
# raw_train['comment_text'] = raw_train['comment_text'].apply(split_comment)

AttributeError: ignored

In [19]:
raw_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"[Explanation, Why, the, edits, made, under, my...",0,0,0,0,0,0
1,000103f0d9cfb60f,"[D'aww!, He, matches, this, background, colour...",0,0,0,0,0,0
2,000113f07ec002fd,"[Hey, man,, I'm, really, not, trying, to, edit...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"["", More, I, can't, make, any, real, suggestio...",0,0,0,0,0,0
4,0001d958c54c6e35,"[You,, sir,, are, my, hero., Any, chance, you,...",0,0,0,0,0,0
